In [1]:
import polars as pl
import polars_ds as pds

In [ ]:
df = pds.random_data(size=2000, n_cols=0).select(
    pds.random(0.0, 1.0).alias("x1"),
    pds.random(0.0, 1.0).alias("x2"),
    pds.random(0.0, 1.0).alias("x3"),
    pds.random_int(0, 3).cast(pl.String).alias("str"),
    pds.random_int(0, 2).alias("target"),
)
df

In [2]:
df = pl.DataFrame({
    "a": [1,2,3,4,5, None],
})

In [3]:
df.select(
    pds.query_longest_streak_below("a", 2).alias("<=2"),
    pds.query_longest_streak_below("a", 6).alias("<=6"),
)

<=2,<=6
i32,i32
2,5


In [ ]:
df = pl.DataFrame({"actual": [1, 0, 1], "pred": [0.4, 0.6, 0.9]})

In [ ]:
df.select(
    pds.query_confusion_matrix("actual", "pred").alias("metrics")
).unnest("metrics")